In [1]:
%pip install accelerate -U


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
dataset = pd.read_excel('dataset.xlsx')

In [4]:
import re

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Удаление ссылок
    text = re.sub(r'<.*?>', '', text)    # Удаление HTML-тегов
    text = re.sub(r'[^а-яА-Яa-zA-Z0-9\s]', '', text)  # Удаление специальных символов
    return text.strip()

# Применение функции
cleaned_data = [clean_text(example) for example in dataset]


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3-8B-Alternate-Tokenizer")

def tokenize_text(text):
    return tokenizer.encode(text, truncation=True, max_length=512)

# Токенизация и нормализация данных
tokenized_data = [tokenize_text(example) for example in cleaned_data]


In [6]:
from sklearn.model_selection import train_test_split

# Разделение данных
train_data, temp_data = train_test_split(tokenized_data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, train_size=0.5, test_size=0.5, random_state=42)


In [16]:
from transformers import TrainingArguments
from transformers import Trainer, AutoModelForCausalLM
import torch
training_args = TrainingArguments(
    output_dir='./finetuned-llama',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    evaluation_strategy='steps',
    eval_steps=500,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
)
MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/home/lockr/.local/share/virtualenvs/GBK_internship-EmzpK5a_/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model-00001-of-00004.safetensors:   3%|2         | 147M/4.98G [00:00<?, ?B/s]

In [ ]:

model = AutoModelForCausalLM.from_pretrained("mymodel")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

# Запуск обучения
trainer.train()


OSError: mymodel does not appear to have a file named config.json. Checkout 'https://huggingface.co/mymodel/tree/None' for available files.

In [ ]:
model.save_pretrained("./model")
tokenizer.save_pretrained("./model")

trainer.evaluate()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

logs_df = pd.DataFrame(logging_callback.logs)



epochs = logs_df['epoch']
eval_loss = logs_df['eval_loss']

plt.figure(figsize=(10, 6))
plt.plot(epochs, eval_loss, marker='o', linestyle='-', color='b', label='Eval Loss')

plt.title('Evaluation Loss vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Eval Loss')
plt.legend()
plt.grid(True)
plt.show()